# Project: Wrangling and Analyzing WeRateDogs' Tweets

## Data Gathering


1. Directly download twitter archive dataset (twitter-archive-enhanced.csv)

In [ ]:
# import modules and load the data 
import pandas as pd
import numpy as np
import matplotlib as plt


%matplotlib inline

twitter_archive = pd.read_csv('twitter-archive-enhanced.csv', sep = ",")

2. Use the Requests library to download the tweet image prediction (image_predictions.tsv)

In [ ]:
# import requests
import requests

# download image predictions file and save it
URL = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
response = requests.get(URL)
open("image_predictions", "wb").write(response.content)

# read image prediction file into Dataframe
image_predictions = pd.read_csv('image_predictions', sep = "\t")

3. Read tweet_json.txt into dataframe

In [ ]:
# read this tweet_json.txt file line by line into a pandas DataFrame 
additional_data = pd.read_json("tweet-json.txt", lines=True)

# create a dataframe that contains tweet id, retweet_count & favourite_count columns
rt_fav_count_df = additional_data[['id', 'retweet_count', 'favorite_count']]

## Assessing Data




In [ ]:
#display twitter archive table


twitter_archive

In [ ]:
# display image predictions table

image_predictions

In [ ]:
# display retweet count and favourite count table

rt_fav_count_df

In [ ]:
# view data info
twitter_archive.info()

In [ ]:
# view data info
image_predictions.info()

In [ ]:
# view data info
rt_fav_count_df.info()

In [ ]:
# view the summary statistics
twitter_archive.describe()

In [ ]:
# view summary statistcs
image_predictions.describe()

In [ ]:
#view summary statistics
rt_fav_count_df.describe()

In [ ]:
# view some random data sample
twitter_archive.sample(5)
image_predictions.sample(5)
rt_fav_count_df.sample(5)

In [ ]:
# check for duplicates across all the data set
all_columns = pd.Series(list(twitter_archive) + list(image_predictions) + list(rt_fav_count_df))
all_columns[all_columns.duplicated()]

### Quality issues

**twitter archive table**
- some ratings are retweet
- tweet ids are saved as integers rather than string 
- insufficient data in column doggo, floofer, pupper and  puppo
- Many missing values in column in_reply_to_status_id, in_reply_to_user_id<br>



**retweet & favourite count table**
- tweet ids saved as integers rather than string 
- zero favourite count despite many retweets
<br>



**image_predictions table**
- image numbers are saved as integer rather than string
- tweet ids are saved as integers rather than string

**Other Quality issues (in the clean phase)** <br>

- two missing rows in retweet count and favourite count columns
- tetweet count and favourite count columns' data types are float rather than integer
- some ratings do not have images <br>




### Tidiness issues
- The text column in twitter archive table contains more than one variable: tweet texts and url
- Retweet count and favourite count columns in `rt_fav_count_df` and `image_prediction` should be part of `twitter_archive`.





## Cleaning Data


In [ ]:
# Make copies of original pieces of data
twitter_archive_clean = twitter_archive.copy()
image_predictions_clean = image_predictions.copy()
rt_fav_count_df_clean = rt_fav_count_df.copy()

### Missing Data
**Insufficient data in column doggo, floofer, pupper and puppo**



#### Define
- Remove columns doggo, floofer, pupper and puppo in twitter archive table because they contain insufficient data

#### Code

In [ ]:
twitter_archive_clean.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis=1, inplace = True)

#### Test

In [ ]:
twitter_archive_clean.columns

**Many missing values in column in_reply_to_status_id, in_reply_to_user_id**

#### Define
- Remove in_reply_to_status_id, in_reply_to_user_id in twitter archive table because the columns have many missing values

#### Code

In [ ]:
twitter_archive_clean.drop(['in_reply_to_status_id', 'in_reply_to_user_id'], axis=1, inplace = True)

#### Test

In [ ]:
twitter_archive_clean.columns


### Tidiness
**The text column in twitter archive table contains more than one variable: tweet texts and url**



### Define
- Extract texts in text column using str.split function

### Code

In [ ]:
twitter_archive_clean['text'] = twitter_archive_clean['text'].str.split('https://', n=1, expand=True)[0]

### Test

In [ ]:
# confirm changes
twitter_archive_clean['text'][9]

**Retweet count and favourite count columns in `rt_fav_count_df` should be part of `twitter_archive`**

### Define

- Merge the `retweet_count` & `favorite_count` columns in `rt_fav_count_clean` to `twitter_archive_clean`, joining on `tweet_id`

### Code

In [ ]:
# first rename 'id' column in Retweet and Favourite Count dataframe to 'tweet_id' 
# to make merging possible 


rt_fav_count_df_clean.rename(columns = {'id':'tweet_id'}, inplace = True)

In [ ]:
# confirm changes
rt_fav_count_df_clean.head(1)

In [ ]:
# merge the columns to twitter archive dataframe

twitter_archive_clean = pd.merge(twitter_archive_clean, rt_fav_count_df_clean,
                            on=['tweet_id'], how='left')

### Test

In [ ]:
# confirm changes
twitter_archive_clean.head(3)

**The columns in `image_predictions` should be part of `twitter_archive`**

### Define

- Merge the columns in image_prediction to twitter_archive_clean, joining on tweet_id

### Code

In [ ]:
twitter_archive_clean = pd.merge(twitter_archive_clean, image_predictions_clean,
                            on=['tweet_id'], how='left')

### Test

In [ ]:
twitter_archive_clean.head(2)

**Two missing rows in retweet count and favorite count columns**

In [ ]:
## view the missing data row

twitter_archive_clean[twitter_archive_clean.retweet_count.isna()]


In [ ]:
## drop rows with missing retweet count and favorite count

twitter_archive_clean.dropna(subset=['retweet_count'], inplace = True)

In [ ]:
# confirm changes
twitter_archive_clean[twitter_archive_clean.favorite_count.isna()]

**Some ratings do not have images**

**Define**
- Drop rows that have ratings but no image

### Code

In [ ]:
# display sample of rows with ratings but no images
twitter_archive_clean[twitter_archive_clean.img_num.isna()].sample(3)

In [ ]:
#drop rows with no image
twitter_archive_clean.dropna(subset=['img_num'], inplace = True)

### Test

In [ ]:
twitter_archive_clean[twitter_archive_clean.img_num.isna()]

### Quality

**Some ratings are retweet**

### Define
- remove rows that have retweets

### Code

In [ ]:
# remove rows that are retweet
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean.retweeted_status_id.isnull()]
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean.retweeted_status_user_id.isnull()]
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean.retweeted_status_timestamp.isnull()]

### Test

In [ ]:
twitter_archive_clean.retweeted_status_timestamp.isnull().all()

In [ ]:
# Lets drop these columns as they are no longer needed

twitter_archive_clean.drop(['retweeted_status_id', 'retweeted_status_user_id','retweeted_status_timestamp'], axis=1, inplace = True)

In [ ]:
twitter_archive_clean.columns

**Image numbers are saved as integer (now float in the master DataFrame) rather than string**

### Define
- convert the image number column's data type from float to string using `astype`

In [ ]:
## change the image number columns data type from float to string

twitter_archive_clean['img_num'] = twitter_archive_clean['img_num'].astype(str)

### Test

In [ ]:

twitter_archive_clean['img_num'].dtype

**Tweet ids are saved as integer rather than string**

### Define

- Convert the tweet id column's data type from a int to a string using `astype`

### Code

In [ ]:
twitter_archive_clean.tweet_id = twitter_archive_clean.tweet_id.astype(str)

### Test

In [ ]:
twitter_archive_clean.tweet_id.dtype

**Retweet count and favorite count  columns' data types are float rather than integer**

### Define

- Convert the Retweet count and favorite count columns' data types from float to int using `astype`

In [ ]:
twitter_archive_clean.retweet_count = twitter_archive_clean.retweet_count.astype('int64')
twitter_archive_clean.favorite_count = twitter_archive_clean.favorite_count.astype('int64')

## Test

In [ ]:
print(twitter_archive_clean.retweet_count.dtype)
print(twitter_archive_clean.favorite_count.dtype)

**Zero favorite count despite many retweets**

### Define
- fill the rows that has zero favourite count by twice the value of corresponding retweet count 

In [ ]:
zero_fav = twitter_archive_clean[twitter_archive_clean['favorite_count'] == 0].index

In [ ]:


twitter_archive_clean.loc[zero_fav,'favorite_count'] = (twitter_archive_clean[twitter_archive_clean['favorite_count'] == 0]['retweet_count'])*2

### Test

In [ ]:
twitter_archive_clean[twitter_archive_clean['favorite_count'] == 0]

## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

In [ ]:
twitter_archive_clean.to_csv('twitter_archive_master.csv', header=True, index=False)

## Analyzing and Visualizing Data


In [ ]:
# load the saved master dataset 

master_data = pd.read_csv('twitter_archive_master.csv')

# display the first five row

master_data.head()

In [ ]:
# convert img_num to string

master_data.img_num = master_data.img_num.astype('int64')
master_data.img_num.dtype

In [ ]:
# view the proportion of image number  

img_num_prop = master_data.img_num.value_counts()/master_data.img_num.value_counts().sum()
img_num_prop

In [ ]:

#plot bar chart

location = [1,2,3,4] 
labels = ['one', 'two', 'three','four']

plt.pyplot.bar(location,img_num_prop, tick_label = labels)


# add a title
plt.pyplot.title('Proportion of Image number', fontsize = 12, weight = 'bold')

#add labels
plt.pyplot.xlabel('Image number', fontsize = 12, weight = 'bold')
plt.pyplot.ylabel('Ratio', fontsize = 12, weight = 'bold');

85.8% of the dog ratings tweets has one image, about 10% has two images, 3.1% of the tweets contains 3 images and about 1.5% has four images. 

In [ ]:
# view the average retweet count group by the number of images 

img_rt_prop = master_data.groupby('img_num').mean()['retweet_count']
img_rt_prop

In [ ]:
#plot bar chart

location = [1,2,3,4] 
labels = ['one', 'two', 'three','four']

plt.pyplot.bar(location,img_rt_prop, tick_label = labels)

# add a title
plt.pyplot.title('Retweet by Image number', fontsize = 12, weight = 'bold')

#add labels
plt.pyplot.xlabel('Image number', fontsize = 12, weight = 'bold')
plt.pyplot.ylabel('Retweet', fontsize = 12, weight = 'bold');

Tweets that contain four images have the highest average retweet count.

In [ ]:
# view the average favourite count group by the number of images 

img_fav_prop = master_data.groupby('img_num').mean()['favorite_count']
img_fav_prop

In [ ]:
#plot bar chart

location = [1,2,3,4] 
labels = ['one', 'two', 'three','four']

plt.pyplot.bar(location,img_fav_prop, tick_label = labels)

# add a title
plt.pyplot.title('Likes by Image number', fontsize = 12, weight = 'bold')

#add labels
plt.pyplot.xlabel('Image number', fontsize = 12, weight = 'bold')
plt.pyplot.ylabel('Like', fontsize = 12, weight = 'bold');

Tweets that contain four images also have the highest favorite count, followed by tweets that contain two images.

In [ ]:
# view the top 5 most featured dog breeds 
# using the #1 prediction i.e p1
master_data.p1.value_counts()[:5]

In [ ]:
# view their combined total as a proportion of the whole featured dogs .
master_data.p1.value_counts()[:5].sum()/master_data.p1.value_counts().sum()

Golden retriever, Labrador retriever, Pembroke, Chihuahua and pug are the most featured dogs in WeRateDog's tweets. Their combined proportion is approxiamtely 23% of the total featured dogs. 

In [ ]:
# display the distribution of dog ratings using rating numerator only. Since the denominator is usually 10

bin = np.arange(0,master_data.rating_numerator.max(),1)
plt.pyplot.hist(data = master_data, x = 'rating_numerator', bins = bin)

# add a title
plt.pyplot.title('Distribution of Dog Ratings', fontsize = 12, weight = 'bold')

#add labels
plt.pyplot.xlabel('Rating numerator', fontsize = 12, weight = 'bold')
plt.pyplot.ylabel('Frequency', fontsize = 12, weight = 'bold')

# set x-axis limit to 17 to zoom in on ratings that occur frequently
plt.pyplot.xlim((0,17));

A typical dog rating on WeRateDog lies between 10/10 and 13/10

### Insights

- 85.8% of the dog ratings tweets has one image, about 10% has two images, 3.1% of the tweets contains 3 images and 1.5% has four images.
- Tweets that contain four images have the highest average retweet count.
- Tweets that contain four images also have the highest favourite count, followed by tweets that contain two images.
- Golden retriever, Labrador retriever, Pembroke, Chihuahua and pug are the most featured dogs in WeRateDogs tweets. Their combined proportion is approximately 23% of the total featured dogs.
- A typical dog rating on WeRateDog lies between 10/10 and 13/10
